In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv
/kaggle/input/milestone2/other/default/1/milstone2.ipynb


In [2]:
import pandas as pd
df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
sample_submission = pd.read_csv('/kaggle/input/System-Threat-Forecaster/sample_submission.csv')
df.head()

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,...,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,0,7.0,0,53447.0,1.0,...,0,0.0,1,0,1.0,0.0,6.0,2018-09-10 10:11:00,2018-04-17,0
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,0,7.0,0,53447.0,1.0,...,1,0.0,0,0,0.0,0.0,10.0,2018-08-16 00:01:00,2018-08-14,1
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,0,7.0,0,53447.0,1.0,...,0,0.0,0,0,0.0,1.0,6.0,2018-09-20 23:20:00,2018-09-11,1
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,0,7.0,0,46413.0,2.0,...,1,0.0,0,0,0.0,0.0,12.0,2018-09-14 00:32:00,2018-01-03,1
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,0,7.0,0,40466.0,2.0,...,0,0.0,0,0,0.0,1.0,7.0,2018-09-15 19:34:00,2018-09-11,0


In [3]:
df.info

<bound method DataFrame.info of                               MachineID   ProductName EngineVersion  \
0      f541bae429089117c4aac39c90dd3416  win8defender   1.1.15200.1   
1      dc2b14d9ce3a0ce4050bb640190f2ca5  win8defender   1.1.15100.1   
2      fd20c5f010e9c5f91ad1c6b3e0da68a0  win8defender   1.1.15200.1   
3      38711eae85eb77a72ec5dfdf27eb2a76  win8defender   1.1.15200.1   
4      32607c9a543a9214e2c7e45800ed4849  win8defender   1.1.15200.1   
...                                 ...           ...           ...   
99995  a378b2196a7a1d8ee0d4f989ee46730e  win8defender   1.1.15200.1   
99996  7f4433ca05d24b9bbdd3932f8b05e61f  win8defender   1.1.15100.1   
99997  d6852dfff534e03d5ddfd3c7875ab072  win8defender   1.1.15200.1   
99998  70a8dc682128c9ff78a3de5aea9dac74  win8defender   1.1.14901.4   
99999  ad9c385b53f2ca788933a39603b84eda  win8defender   1.1.15000.2   

             AppVersion SignatureVersion  IsBetaUser  RealTimeProtectionState  \
0       4.18.1807.18075     1.275.

In [4]:
#df =df.dropna(subset=['target'])
#df.info()

In [5]:
from sklearn.impute import SimpleImputer

# Identify numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = num_cols.drop('target', errors='ignore')

# Impute numerical columns with mean
imputer_num = SimpleImputer(strategy='mean')
df[num_cols] = imputer_num.fit_transform(df[num_cols])
test_data[num_cols] = imputer_num.transform(test_data[num_cols])

# Impute categorical columns with most frequent value
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
test_data[cat_cols] = imputer_cat.transform(test_data[cat_cols])

print("Missing values handled successfully!")

Missing values handled successfully!


In [6]:
df['DateAS'] = pd.to_datetime(df['DateAS']).dt.month
df['DateOS'] = pd.to_datetime(df['DateOS']).dt.month
test_data['DateAS'] = pd.to_datetime(test_data['DateAS']).dt.month
test_data['DateOS'] = pd.to_datetime(test_data['DateOS']).dt.month

In [7]:
cardinality = df.nunique().sort_values(ascending=False)

low_cardinality = cardinality[cardinality == 1].index.tolist()
df = df.drop(columns=low_cardinality)
test_data = test_data.drop(columns=low_cardinality)
df.shape

(100000, 73)

In [8]:
categorical_cols = df.select_dtypes(include=['object']).columns

selected_features = [col for col in categorical_cols if df[col].nunique() <= 10]
high_cardinality_features = [col for col in categorical_cols if df[col].nunique() > 10]
df = pd.get_dummies(df, columns=selected_features, drop_first=True)
test_data= pd.get_dummies(test_data, columns=selected_features, drop_first=True)
df

,MachineID,EngineVersion,AppVersion,SignatureVersion,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,...,LicenseActivationChannel_Retail,LicenseActivationChannel_Retail:TB:Eval,LicenseActivationChannel_Volume:GVLK,LicenseActivationChannel_Volume:MAK,FlightRing_NOT_SET,FlightRing_RP,FlightRing_Retail,FlightRing_Unknown,FlightRing_WIF,FlightRing_WIS
0,f541bae429089117c4aac39c90dd3416,1.1.15200.1,4.18.1807.18075,1.275.1003.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
1,dc2b14d9ce3a0ce4050bb640190f2ca5,1.1.15100.1,4.18.1807.18075,1.273.1465.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,1.1.15200.1,4.18.1807.18075,1.275.1546.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
3,38711eae85eb77a72ec5dfdf27eb2a76,1.1.15200.1,4.12.17007.18011,1.275.1141.0,7.0,0.0,46413.0,2.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
4,32607c9a543a9214e2c7e45800ed4849,1.1.15200.1,4.13.17134.228,1.275.1283.0,7.0,0.0,40466.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,a378b2196a7a1d8ee0d4f989ee46730e,1.1.15200.1,4.18.1807.18075,1.275.1582.0,7.0,0.0,68585.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
99996,7f4433ca05d24b9bbdd3932f8b05e61f,1.1.15100.1,4.18.1806.18062,1.273.1156.0,7.0,0.0,11280.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
99997,d6852dfff534e03d5ddfd3c7875ab072,1.1.15200.1,4.18.1807.18075,1.275.209.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
99998,70a8dc682128c9ff78a3de5aea9dac74,1.1.14901.4,4.16.17656.18052,1.269.641.0,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,True,False,False


In [9]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in high_cardinality_features:
    # Apply Label Encoding
    df[col] = encoder.fit_transform(df[col])
    test_data[col] = encoder.fit_transform(test_data[col]) 

# Display the transformed dataset
df.head()

,MachineID,EngineVersion,AppVersion,SignatureVersion,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,...,LicenseActivationChannel_Retail,LicenseActivationChannel_Retail:TB:Eval,LicenseActivationChannel_Volume:GVLK,LicenseActivationChannel_Volume:MAK,FlightRing_NOT_SET,FlightRing_RP,FlightRing_Retail,FlightRing_Unknown,FlightRing_WIF,FlightRing_WIS
0,95607,34,31,2218,7.0,0.0,53447.0,1.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False
1,85800,33,31,1906,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
2,98689,34,31,2331,7.0,0.0,53447.0,1.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
3,21877,34,17,2253,7.0,0.0,46413.0,2.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
4,19567,34,22,2265,7.0,0.0,40466.0,2.0,1.0,1.0,...,True,False,False,False,False,False,True,False,False,False


In [10]:
num_cols = df.select_dtypes(include=['int64','int32','float64']).columns.drop('target', errors='ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
test_data[num_cols] = scaler.fit_transform(test_data[num_cols]) 

In [11]:
df= df.drop(columns=list(set(df.columns)-(set(test_data.columns)|{'target'})), axis=1)

In [12]:
from sklearn.preprocessing import LabelEncoder

# Automatically identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
test_categorical_cols = test_data.select_dtypes(include=['object']).columns

# Apply Label Encoding
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])
for col in test_categorical_cols:
    test_data[col] = encoder.fit_transform(test_data[col])    

In [13]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['target'])   # Features
y = df['target']     # Target column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize the XGBoost classifier
xgb = XGBClassifier(n_estimators=500, random_state=42)

# Train the model
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6134


In [15]:
test_data = test_data[X.columns]

In [16]:
#Q1 = df.target.quantile(0.25)
#Q3 = df.target.quantile(0.75)
#IQR = Q3 - Q1
#df = df[(df.target >= (Q1 - 1.5*IQR)) & (df.target <= (Q3+1.5*IQR))]

In [17]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
model = GradientBoostingRegressor(
    n_estimators=550,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    subsample=0.8,
    max_features=0.8,
    loss='huber',
    alpha=0.75,
)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
# Train performance
train_r2 = r2_score(y_train, y_pred_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

# Test performance
test_r2 = r2_score(y_test, y_pred_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Train R²: {train_r2}")
print(f"Test R²: {test_r2}")
print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")
print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

Train R²: 0.15471716969612193
Test R²: 0.06256203899530544
Train MAE: 0.4095271149876729
Test MAE: 0.43389510589320934
Train MSE: 0.21129925733547153
Test MSE: 0.23432460818464465


In [18]:
#import pandas as pd
#df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#df_copy = df.copy()

In [19]:
#constant_columns = df.nunique()[df.nunique() == 1]
#print(constant_columns)

In [20]:
#import pandas as pd
#df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#sample_submission = pd.read_csv('/kaggle/input/System-Threat-Forecaster/sample_submission.csv')
#df.head()


In [21]:
#df =df.dropna(subset=['target'])
#df.info()



In [22]:
#Q1 = df.target.quantile(0.25)
#Q3 = df.target.quantile(0.75)
#IQR = Q3 - Q1
#df = df[(df.target >= (Q1 - 1.5*IQR)) & (df.target <= (Q3+1.5*IQR))]

In [23]:
#from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#model = GradientBoostingRegressor(
  #  n_estimators=550,
   # learning_rate=0.05,
  #  max_depth=5,
  #  min_samples_split=10,
   # min_samples_leaf=5,
  #  subsample=0.8,
  #  max_features=0.8,
  #  loss='huber',
  #  alpha=0.75,
#)
#model.fit(X_train, y_train)
#y_pred_train = model.predict(X_train)
#y_pred_test = model.predict(X_test)
# Train performance
#train_r2 = r2_score(y_train, y_pred_train)
#train_mae = mean_absolute_error(y_train, y_pred_train)
#train_mse = mean_squared_error(y_train, y_pred_train)
#
# Test performance
#test_r2 = r2_score(y_test, y_pred_test)
#test_mae = mean_absolute_error(y_test, y_pred_test)
#test_mse = mean_squared_error(y_test, y_pred_test)
#
#print(f"Train R²: {train_r2}")
#print(f"Test R²: {test_r2}")
#print(f"Train MAE: {train_mae}")
#print(f"Test MAE: {test_mae}")
#print(f"Train MSE: {train_mse}")
#print(f"Test MSE: {test_mse}")

In [24]:
test_predictions = model.predict(test_data)  # Generate predictions

# Create submission DataFrame (Ensure sample_submission exists)
submission = sample_submission.copy()
submission['target'] = test_predictions  

# If classification-like labels are needed (Assuming binary case)
# Ensure model output is properly rounded or thresholded
submission['target'] = submission['target'].apply(lambda x: '1' if x > 0.5 else '0')

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Output file ready for submission
print("Submission file created.")

Submission file created.


In [25]:
#if 'PrimaryDisplayResolutionHorizontal' in df.columns and 'PrimaryDisplayResolutionVertical' in df.columns:
    #resolution_count = df[(df['PrimaryDisplayResolutionHorizontal'] == 1366) & (df['PrimaryDisplayResolutionVertical'] == 768)].shape[0]
   # print(f'Number of systems with a screen resolution of 1366 x 768: {resolution_count}')
#else:
  #  print('Columns for horizontal and vertical resolution not found in the dataset.')

In [26]:
# Filter the dataset for gamers with malware detected
#gamer_malware_count = df[(df['IsGamer'] == 1) & (df['target'] == 1)].shape[0]

#print(f'Number of systems owned by gamers where malware was detected: {gamer_malware_count}')

In [27]:
#df =pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")


In [28]:
#X_test =pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")

In [29]:
#import pandas as pd  


#df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")


#df_clean = df.dropna()





In [30]:
#X =df.drop("target", axis =1)
#y =df["target"]

In [31]:
#from sklearn.dummy import DummyClassifier
#model =DummyClassifier().fit(X,y)
#y_pred =model.predict(X_test)

In [32]:
#submission =pd.DataFrame({"id": range(0,X_test.shape[0]),

#"target": y_pred})

#submission.to_csv('submission.csv', index =False)


In [33]:
#import pandas as pd  

# Load the dataset
#train_df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")

# Drop rows with missing values
#train_clean = train_df.dropna()

# Now, you can proceed with calculations
#max_antivirus = train_clean["NumAntivirusProductsInstalled"].max()
#print("Max NumAntivirusProductsInstalled:", max_antivirus)


In [34]:
#if "OSVersion" in train_clean.columns:
   # unique_os_versions = train_clean["OSVersion"].nunique()
  #  print("Unique Operating System Versions:", unique_os_versions)
#else:
   # print("Column 'OperatingSystemVersion' not found. Check column names.")
    

In [35]:
#median_ram = train_clean["TotalPhysicalRAMMB"].median()
#print("50th Percentile (Median) TotalPhysicalRAMMB:", median_ram)


In [36]:
#most_frequent_rtps = train_clean[train_clean["IsPassiveModeEnabled"] == 1]["RealTimeProtectionState"].mode()[0]
#print("Most Frequent RealTimeProtectionState when IsPassiveModeEnabled = 1:", most_frequent_rtps)


In [37]:
#import matplotlib.pyplot as plt  
#import seaborn as sns  

# Distribution of RAM
#plt.figure(figsize=(8, 5))
#sns.histplot(train_clean["TotalPhysicalRAMMB"], bins=30, kde=True)
#plt.title("Distribution of Total Physical RAM (MB)")
#plt.show()


In [38]:
#plt.figure(figsize=(10, 5))
#sns.heatmap(train_df.isnull(), cmap="viridis", cbar=False, yticklabels=False)
#plt.title("Missing Values in the Dataset")
#plt.show()


In [39]:
#plt.figure(figsize=(12, 6))
#sns.countplot(data=train_clean, x="OSVersion", order=train_clean["OSVersion"].value_counts().index[:10])
#plt.xticks(rotation=90)
#plt.title("Top 10 Operating System Versions in Dataset")
#plt.show()
